In [ ]:
import numpy as np
import os
from tifffile import imread, imwrite
from shutil import rmtree
from tqdm.notebook import trange
import subprocess
import glob
from tqdm import tqdm
import time
import roifile

In [ ]:
def normalize8(I):
    
    # use either global estimate or manual threshold
    mx = I.max() 
    # mx = 10000

    normalized = I / mx
    normalized[normalized>1.0] = 1.0
    new = normalized * 255.0
    return new.astype(np.uint8) # 8-bit image is required for MitoGraph

In [ ]:
# the bottom and top z for ROIs, get from the image
low_z, high_z = None, None

In [ ]:
# crop ROI at each timepoint into a folder for mitograph
data_path = './tif_folder/'
save_path = './segmented_data/'
if not os.path.isdir(save_path):
    os.makedirs(save_path)

# load prepared ROI files
roi_path = './ROI/'
roi_names = os.listdir(roi_path) # make sure this folder has individual .roi files
rois = [roifile.ImagejRoi.fromfile(roi_path+name) for name in roi_names if '.roi' in name]
    
mito_tif_list = os.listdir(data_path)

for frame in trange(len(mito_tif_list)): # loop all timepoints
# for frame in trange(1): # test first frame (recommended for first time, check first.)

    full_data = imread(data_path+mito_tif_list[frame])

    roi_id = 1
    for index, roi in enumerate(rois):
        roi_id_dir = save_path+'roi_id_'+str(roi_id)+'/'
        if not os.path.isdir(roi_id_dir):
            os.mkdir(roi_id_dir)
            os.mkdir(roi_id_dir+'mitograph/')
        zstart,zend,ystart,yend,xstart,xend = [low_z,high_z,roi.top,roi.bottom,roi.left,roi.right]

        cropped_data = full_data[zstart:zend, ystart:yend, xstart:xend]
        cropped_data = normalize8(cropped_data)

        # make dir and save for each frame
        frame_dir = roi_id_dir+'mitograph/frame_'+str(frame)+'/'
        if not os.path.isdir(frame_dir):
            os.mkdir(frame_dir)

        print('Writing ROI', roi_id)
        imwrite(frame_dir+'frame_'+str(frame)+'.tif', cropped_data)
        
        roi_id += 1